Rename a bunch of files using Git and renamer?



In [ ]:
var renamer = require('renamer');

function renameUsingGit(gitRoot, match, find, replace) {
    var files = renamer.expand(path.join(gitRoot, match));
    var results = renamer.replace({
        files: files.filesAndDirs,
        find: find,
        replace: replace
    });
    return renamer.dryRun(results).list.map(r => {
        // rename with git instead
        var output = execSync('git '
            + '--work-tree=' + JSON.stringify(gitRoot)
            + ' --git-dir=' + JSON.stringify(path.join(gitRoot, '.git'))
            + ' mv ' + JSON.stringify(r.before)
            + ' ' + JSON.stringify(r.after));
        return output.toString() || r.after;
    });
};
renameUsingGit;


In [ ]:
// test it
// rename all scss files to less
//renameUsingGit(
//    '/Users/briancullinan/Documents/studysauce4',
//    'src/**/*.css',
//    '.css',
//    '.scss');


json git tree?


In [ ]:
var execSync = require('child_process').execSync;
/*var {
    execCmd
} = importer.import([
    'spawn child process',
]);*/


function gitTree(project) {
    console.log('creating tree "' + project + '"');
    var branches = execSync('git branch --list --all ', {cwd: project}).toString()
        .trim().split(/\s*\n\s*\**\s*/ig)
        .map(b => b.replace('remotes/', '').trim());

    // reverse git so we can add merges to the "graph" after the branch tree is constructed
    var branch = execSync('git log --pretty=format:"%ct %d %h %p" '
        + '--reverse --branches --tags ', {cwd: project}).toString().split(/\s*\n\s*/ig);
    var all = []; //execSync('git log --pretty=format:"%h %p" --reverse --all').toString().split(/\s*\n\s*/ig);

    var nodes = [];
    var edges = [];
    branch.forEach(b => {
        var commits = ((/[0-9]+\s+\(.*?\)\s*(.+)/ig).exec(b)
            || (/[0-9]+\s+(.+)/ig).exec(b) || [b, b.trim()])[1].trim().split(/\s+/ig);
        commits.forEach(r => {
            if (nodes.filter(d => d.name == r).length == 0) {
                nodes[nodes.length] = {
                    name: r,
                    index: nodes.length,
                    branch: ((/[0-9]+\s+\((.*?)\)/ig).exec(b) || [b, ''])[1],
                    time: (/([0-9]+)\s+/ig).exec(b)[1]
                }
            }
        });
        // don't include merges in tree
        if (commits.length == 3) {
            commits = [commits[0], commits[1]];
        }
        if (commits.length == 2) {
            edges = edges.concat(commits.slice(1)
                // check all existing sources to test for merge
                .map((c, i) => {
                    return {source: c, target: commits[i]}; // -1 because of slice(1)
                }))
        }
    });

    return formatNodes(nodes, edges.reverse());
}
module.exports = gitTree;



tip git tree?

just the tip?

source tree?



In [2]:
var importer = require('../Core');
var _ = require('underscore');
var fs = require('fs');
var path = require('path');
var {
    formatNodes, gitTree, displayBranches,
    formatNodes, cloneProject, execCmd
} = importer.import([
    'spawn child process',
    'd3.ipynb[format d3 tree]',
    'json git tree',
    'd3.ipynb[display d3 tree]',
    'git project directory'
]);

// TODO: convert this to bash
function getResetCommit(branch, parent, project) {
    branch = branch.replace('HEAD -> ', '').trim();
    parent = parent.replace('HEAD -> ', '').trim();
    return execCmd(`
git checkout ${branch}
git reset --mixed ${parent}
git add -A
git commit -m "auto reset ${branch}"
`, {cwd: project});
}

var svgs = [];
function resetAllBranches() {
    var children = [];
    var nodeTree = _.sortBy(gitTree(project), a => a.time);
    var promises = [];
    for (const n of nodeTree) {
        var current = n;
        var parentCount = 0;
        while (typeof current.parent !== 'undefined'
            && current.parent.branch === current.branch) {
            current = current.parent;
            parentCount++;
        }
        if (parentCount > 0 && typeof current.parent !== 'undefined') {
            // TODO: keep track of the parent so the squashed branch can be merged on to the newly squashed parent
            promises.push(((current, n) => resolve => {
                if (typeof n.children !== 'undefined') {
                    children.push(n);
                }
                console.log(current.parent.name);
                return getResetCommit(n.branch, current.parent.name, project)
                    .then(() => {
                        var nodeTree = _.sortBy(gitTree(project), a => a.time);
                        svgs.push(displayBranches(nodeTree));
                        resolve();
                    })
            })(current, n));
        }
    }
    return importer.runAllPromises(promises)
        .then(() => children);
}

function gitTipOfTree(project) {
    svgs = [];
    return cloneProject(project, true)
        // TODO: git checkout command
        // TODO: pull all missing remmotes
        .then(r => {
            project = r;
            var nodeTree = _.sortBy(gitTree(project), a => a.time);
            const head = nodeTree.filter(r => r.branch.indexOf('HEAD') > -1)[0];
            const branch = head
                ? (nodeTree.filter(r => r.branch.indexOf(head.branch.replace(/HEAD *-*>* */ig, '').split(/\s*,\s*/ig)) > -1)[0] || nodeTree[0])
                : nodeTree[0];
            return displayBranches([branch]);
        })
        .then(r => svgs.push(r))
        //.then(() => resetAllBranches())
        .then(() => svgs)
};
module.exports = gitTipOfTree;


reading notebook /Users/briancullinan/jupytangular2/Utilities/git.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/git.ipynb
compiling /Users/briancullinan/jupytangular2/Utilities/child process.ipynb[0]
compiling /Users/briancullinan/jupytangular2/Frameworks/d3.ipynb[4]
compiling /Users/briancullinan/jupytangular2/Utilities/git.ipynb[2]
compiling /Users/briancullinan/jupytangular2/Frameworks/d3.ipynb[5]
compiling /Users/briancullinan/jupytangular2/Utilities/git.ipynb[7]
compiling /Users/briancullinan/jupytangular2/Frameworks/gulp.ipynb[1]
compiling /Users/briancullinan/jupytangular2/Utilities/file system.ipynb[7]


[Function: gitTipOfTree]

In [ ]:
var importer = require('../Core');
var gitTipOfTree = importer.import('git.ipynb[tip git tree]');

$$.async();
gitTipOfTree('https://github.com/Soundnode/soundnode-app')
    .then(r => $$.html(r.join('')))
    .catch(e => $$.sendError(e))


compiling /Users/briancullinan/jupytangular2/Utilities/git.ipynb[3]
compiling /Users/briancullinan/jupytangular2/Utilities/child process.ipynb[0]
compiling /Users/briancullinan/jupytangular2/Frameworks/d3.ipynb[4]
compiling /Users/briancullinan/jupytangular2/Utilities/git.ipynb[2]
compiling /Users/briancullinan/jupytangular2/Frameworks/d3.ipynb[5]
compiling /Users/briancullinan/jupytangular2/Utilities/git.ipynb[7]
compiling /Users/briancullinan/jupytangular2/Frameworks/gulp.ipynb[1]
compiling /Users/briancullinan/jupytangular2/Utilities/file system.ipynb[7]
spawning process git clone -v https://github.com/Soundnode/soundnode-app


Cloning into 'soundnode-app'...

POST git-upload-pack (gzip 1342 to 750 bytes)



Display graphs of different commands and scenarios using dry-run?

https://github.com/kpdecker/jsdiff

http://incaseofstairs.com/jsdiff/



In [ ]:
var path = require('path');
var gulp = require('gulp');
var run = require('gulp-run');
var tap = require('gulp-tap');
var importer = require('../Core');
var gulpPromise = importer.import('gulp task to a Promise', {gulp})

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = typeof project === 'undefined' ? path.join(PROFILE_PATH, 'Documents/portal') : project;

gulp.task('git watch', function () {
    return run('git diff -w a229417..498d5a5')
        .pipe(tap(file => console.log(file)))
        .pipe(gulp.dest('output'));
});

function getBranchBoundaries(p) {
    process.chdir(project)
    project = p;
    return gulpPromise(['git watch'])
}
module.exports = getBranchBoundaries;

if(typeof $$ !== 'undefined') {
    $$.async();
    getBranchBoundaries(project)
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e));
}



check github for commits since the current one?



In [ ]:
var _ = require('lodash');
var importer = require('../Core');
var { gitTree, execCmd, getOauthClient, updateEvent, cloneProject } = importer.import([
    'json git tree',
    'spawn child process',
    'import google calendar api',
    'create merge delete event',
    'git project directory'
]);

var options = {
    calendarId: 'aws'
}

// create a calendar event to retest this branch
function installBuildTestEvent() {
    const now = new Date();
    const event = {
        summary: 'spawn child process',
        description: JSON.stringify({
            script: `npm install\nnpm run build\nnpm run test`,
            options: {cwd: project} // TODO: fix current working directory using project name
        }, null, 4),
        start: {dateTime: new Date(now.getTime())},
        end: {dateTime: new Date(now.getTime() + 30 * 60 * 1000)}
    }
    return getOauthClient(options)
        .then(() => updateEvent(event, options))            
}

function findBranchRemote(project, branch) {
    var head = branch || '';
    var nodeTree = _.sortBy(gitTree(project), a => a.time);
    for (const n of nodeTree) {
        if(n.branch.indexOf('HEAD') > -1) {
            // try to find a remote on the same branch
            console.log(n.branch);
            head = n.branch.split(/\s*,\s*|\s*->\s*/igm)[1].trim();
            //console.log(n);
            break;
        }
    }
    // TODO: import commands from git commands bash notebook instead
    if(head.length === 0) {
        throw new Error('head not found, specify branch');
    }
    let remote;
    return execCmd(`git branch -r`, {cwd: project}) // 
        .then(r => {
            const remotes = r.join('\n').split(/\s*\n\s*/igm).filter(r => (r.split('/')[1] || '').trim() === head);
            if(remotes.length === 0) {
                throw new Error('branch "' + head + '" not found on any remotes ' + JSON.stringify(r))
            }
            remote = remotes[0].split('/')[0];
            return remote;
        })
        .then(() => ({remote, branch}))
}

function getUpdate(project, branch) {
    return cloneProject(project)
        .then(r => {
            project = r;
            return findBranchRemote(project, branch)
        })
        .then(({remote, branch}) => execCmd(`
git fetch ${remote}
git log HEAD..${remote}/${branch}`, {cwd: project})
            .then(log => {
                if(log.join('').trim().length > 0) {
                    return execCmd(`
git reset --hard
git pull ${remote} ${branch} --rebase`, {cwd: project})
                         .then(() => installBuildTestEvent())
                } else {
                    throw new Error('nothing for git to do');
                }
            }))
}

// git log currentbranch@{1}..currentbranch
// where the @{1} notation means "the commit the branch pointed to just before it last got updated".
// git log HEAD..swiftpage/development

if(typeof $$ !== 'undefined') {
    $$.async();
    getUpdate('portal')
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


TODO:

Rebase squash cherrypick automatically to create perfect feature branch?

Reset each file one at a time to make sure there aren't any unnecessary changes?

Use unit testing to warn about unnecessary modules?

Load relevant projects based on branch.  It could parse the .sln file and automatically patch whenever the checked out index changes.

Plot when a TODO: item is added and when it is removed on Google Calendar, then verify by checking commit -messages, remaining comments/function names with matching keywords (split by camel casing)?, plot dates completed for measurement of self set tasks and short term goals, which activities are harder than others?  better insight in to erroneous developer time estimates?

Interface for searching all TODO: items?  Calendar?



git project directory?

get fresh git project directory?


In [ ]:
var gulp = require('gulp');
var copy = require('gulp-copy');
var clean = require('gulp-clean');
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var { listInProject, tasksToPromise } = importer.import([
    'gulp tasks',
    'list project files'
], {gulp});

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';

gulp.task('clear project', function () {
    return gulp.src(project, {read: false})
        .pipe(clean({force: true}));
});

gulp.task('copy index', function () {
    return gulp.src(project_orig)
        .pipe(copy(project))
        .dest(project);
});

function deepCopy(project_orig) {
    const now = new Date();
    const projectName = path.basename(project_orig) + '-copy-' + now.getFullYear() + '-' + (now.getMonth() + 1) + '-' + now.getDate() + '-' + now.getHours() + '-' + now.getMinutes() + '-' + now.getSeconds();
    const project = path.join(path.dirname(project_orig), projectName);
    // TODO: delete old cloned directories
    /* fs.existsSync(project) ? tasksToPromise(['clear project']) :*/
    // do a deep copy of all git files
    return Promise.all([
        listInProject(project_orig, '**/*'),
        listInProject(project_orig + '/.git', '**/*')
    ])
        // TODO: clone from remote
        .then(r => {
            console.log('cloning repo ' + project_orig + ' -> ' + project);
            for (const file of [].concat(...r).sort()) {
                const dirname = path.dirname(
                    file.replace(project_orig, project)).split('/');

                for (var i = 2; i <= dirname.length; i++) {
                    if (!fs.existsSync(dirname.slice(0, i).join('/'))) {
                        fs.mkdirSync(dirname.slice(0, i).join('/'));
                    }
                }
                fs.copyFileSync(file, file.replace(project_orig, project));
            }
        })
        .then(() => project)
}

function cloneProject(project, copy) {
    if(!project) {
        throw new Error('specify project path or github');
    }
    if(fs.existsSync(path.resolve(project))) {
        project = path.resolve(project);
    } else if(fs.existsSync(path.join(PROFILE_PATH, 'Documents/' + project))) {
        project = path.resolve(path.join(PROFILE_PATH, 'Documents/' + project));
    } else if(fs.existsSync(path.join(PROFILE_PATH, project))) {
        project = path.resolve(path.join(PROFILE_PATH, project));
    } else if(project.match(/github.com\/[a-z0-9_-]+\/[a-z0-9_-]+/ig)) {
        // fetch remote
        const projectName = (/github.com\/([a-z0-9_-]+)\/([a-z0-9_-]+)/ig).exec(project)[2];
        const projectHttp = project.replace(/^(https*)*:*\/*\/*/ig, 'https://');
        if(fs.existsSync(path.join(PROFILE_PATH, projectName))) {
            project = path.resolve(path.join(PROFILE_PATH, projectName));
        } else {
            return execCmd(`
git clone -v ${projectHttp}
`, {cwd: path.resolve(PROFILE_PATH)})
                .then(r => path.resolve(path.join(PROFILE_PATH, projectName)))
                .then(r => copy ? deepCopy(r) : r)
        }
    } else {
        throw new Error('invalid project path "' + project + '"');
    }
    return Promise.resolve(project)
            .then(r => copy ? deepCopy(r) : r)
}
module.exports = cloneProject;

